# 지진해일대피소 현황 분석 및 개선 방안 제시

In [1]:
import pandas as pd 
import folium 
import matplotlib
import matplotlib.pyplot as plt
import pylab
import numpy as np
from folium import Map


#설치 안된거는 설치해주세요

In [2]:
# 파이그래프에서 한국어 나오게 하는 코드 

import matplotlib.font_manager as fm

for font in fm.fontManager.ttflist:
    if font.name in ['AppleGothic', 'Malgun Gothic']:
        plt.rcParams['font.family'] = font.name
        break

plt.rcParams['font.family']

['AppleGothic']

# 선택한  데이터 
- 전국지진해일대피소표준데이터
    - 통합, 광명시 
- KOSIS 행정구역(시군구)별 인구 통계 데이터 

# EDA

## 서울특별시, 인천광역시, 경기도 데이터만 추출
### 특이사항
- 소재지도로명주소와 소재지지번주소 둘 중 하나만 기입되어 도로명 주소 결측치는 소재지지번주소 컬럼에서 가져옴
- 경기도 광명시 데이터가 몇일 전에 올라와서 따로 다운받아 전국 통합본에 합침

### 과정
1. 주소를 string type으로 변환
2. 띄어쓰기를 기준으로 첫번째 덩어리 ('서울특별시', '인천광역시', '경기도' 등등)만 'area' 컬럼으로 저장 
3. area가 '서울특별시', '인천광역시', '경기도'인건만 새로운 테이블(shelter_GG)로 저장 

In [3]:
shelter = pd.read_csv('전국지진해일대피소표준데이터.csv', encoding = "CP949")

In [4]:
len(shelter)

10305

In [5]:
kwangmyung = pd.read_csv('경기도_광명시_지진해일대피소_20190909.csv', encoding = "CP949")

FileNotFoundError: [Errno 2] File b'\xea\xb2\xbd\xea\xb8\xb0\xeb\x8f\x84_\xea\xb4\x91\xeb\xaa\x85\xec\x8b\x9c_\xec\xa7\x80\xec\xa7\x84\xed\x95\xb4\xec\x9d\xbc\xeb\x8c\x80\xed\x94\xbc\xec\x86\x8c_20190909.csv' does not exist: b'\xea\xb2\xbd\xea\xb8\xb0\xeb\x8f\x84_\xea\xb4\x91\xeb\xaa\x85\xec\x8b\x9c_\xec\xa7\x80\xec\xa7\x84\xed\x95\xb4\xec\x9d\xbc\xeb\x8c\x80\xed\x94\xbc\xec\x86\x8c_20190909.csv'

In [ ]:
len(kwangmyung)

In [ ]:
pd.DataFrame(kwangmyung.isnull().sum())

In [ ]:
kwangmyung['제공기관명'] = '경기도 광명시'

In [ ]:
shelter = pd.concat([shelter, kwangmyung], ignore_index = True, sort = False)

In [ ]:
len(shelter)

In [ ]:
shelter.columns

# 위도 경도 결측치 채우기 
 
1. 위도, 경도 결측치인 행만 빼서 하나의 테이블(nulls)로 저장 
    - 정해진 API이용횟수 때문
2. KAKAO API와 LOCATIONIQ API 활용해서 위도 경도 찾음 
3. 두 API로도 안나온 주소지는 손수 찾아서 채움 
4. 본래 통합본에 결측치 테이블(nulls) LEFT JOIN해서 최종 테이블(shelter_G)로 저장
5. 위도, 경도 결측치 남은 것 없는지 확인

In [ ]:
shelter[shelter['소재지도로명주소'].isnull()]

In [ ]:
len(shelter[shelter['소재지도로명주소'].isnull()])

In [ ]:
len(shelter[shelter['소재지도로명주소'].isnull()&shelter['소재지지번주소'].isnull()])

In [ ]:
shelter[shelter['소재지도로명주소'].isnull()].head()

In [ ]:
shelter['소재지도로명주소'] = shelter['소재지도로명주소'].fillna(shelter['소재지지번주소'])

In [ ]:
shelter[shelter['소재지도로명주소'].isnull()]

In [ ]:
shelter['소재지도로명주소'][1103]

In [ ]:
# 소재지도로명주소 string타입으로 변형 
shelter['소재지도로명주소'] = shelter['소재지도로명주소'].astype('str')

In [ ]:
# 소재지도로명 띄어쓰기 기준으로 잘라 첫번째 덩어리만 area 컬럼에 저장
shelter['area'] = shelter['소재지도로명주소'].str.split(' ').str[0]

In [ ]:
shelter.head()

In [ ]:
shelter.area.unique()

In [ ]:
shelter_GG = shelter[(shelter['area']=='서울특별시')|(shelter['area']=='인천광역시')|(shelter['area']=='경기도')]

In [ ]:
shelter_GG.area.unique()

In [ ]:
len(shelter_GG)

In [ ]:
# 컬럼별 null값 찾기
data = shelter_GG.isnull().sum()
pd.DataFrame(data)

In [ ]:
nulls = shelter_GG[shelter_GG['위도'].isnull()]

In [ ]:
# from geocoder import geocode

In [ ]:
# lats = []
# lngs = []

# for i, index in enumerate(nulls.index):
#     print(i + 1, '-', index)
#     lat, lng = geocode(nulls['소재지도로명주소'][index])
#     lats.append(lat)
#     lngs.append(lng)

# nulls = nulls.assign(lat=lats, lng=lngs)

# nulls.head()



In [ ]:
# nulls.to_csv('nulls.csv')

In [ ]:
# import os
# import requests

In [ ]:
# LOCATIONIQ_TOKEN = os.environ['LOCATIONIQ_TOKEN']

In [ ]:
# def gc(address):
#     url = 'https://us1.locationiq.com/v1/search.php'
#     payload = {
#         'key': LOCATIONIQ_TOKEN,
#         'q': address,
#         'format': 'json'
#     }
#     response = requests.get(url, params=payload)
#     results = response.json()
#     if 'error' in results:
#         return None, None
#     coordinates = results[0]
#     return coordinates['lat'], coordinates['lon']

In [ ]:
# lats = []
# lngs = []

# for i, index in enumerate(nulls.index):
#     print(i + 1, '-', index)
#     lat, lng = gc(nulls['소재지도로명주소'][index])
#     lats.append(lat or 0)
#     lngs.append(lng or 0)

# nulls = nulls.assign(위=lats, 경=lngs)

# nulls.head()

In [ ]:
nulls = pd.read_csv('nulls.csv')

In [ ]:
nulls.head()

In [ ]:
shelter_G = shelter_GG.merge(nulls, how='left', on='지진해일대피소명')

In [ ]:
len(shelter_G) #merge 잘 됐는지 확인 완료

In [ ]:
 #nulls 테이블에 있던 위도 경도 shelter_GG 테이블 위도 경도로 옮기기
shelter_G['위도'] = shelter_G['위도'].fillna(shelter_G['위'])

In [ ]:
shelter_G['경도'] = shelter_G['경도'].fillna(shelter_G['경'])

In [ ]:
pd.DataFrame(shelter_G.isnull().sum()) #위도, 경도 결측치 다 채운 것 확인완료 

# 분석 시작

## 1. 지진해일대피소유형 분석
- 옥외대피소 vs 실내대피소

In [ ]:
shelter_GG['지진해일대피소유형'].unique()

In [ ]:
labels=['옥외대피소', '실내대피소']
shelter_GG['지진해일대피소유형'].value_counts().plot.pie(labels = labels
                    ,colors= ['#98DBC6', '#FFCCBB']
                    ,startangle = 60
                    ,shadow = True
                    ,autopct = '%1.1f%%'
                    ,explode = (0,0.1) 
                    ,figsize=(5, 5)
                    ,fontsize = 15)
plt.title('경기도 지진해일대피소구분', fontsize=20)
pylab.ylabel('')

## 2. 내진설계 분석
- 실내 대피소 중 내질설계가 되어있는 대피소는 얼마나 될까?


In [ ]:
for_seismic = shelter_G[shelter_G['지진해일대피소유형']=='실내대피소']

# 실내 대피소만 내진설계가 가능하기 때문에 실내 대피소만 

In [ ]:
seismic_design = len(for_seismic[for_seismic['내진적용여부']=='적용'])/len(for_seismic)*100
print('경인, 경기지역 내진설계 비율 =', seismic_design, '%')

In [ ]:
labels=['적용', '미적용']   #항상 파이그래프 보면서 label순서 제대로 설정했는지 확인 
for_seismic['내진적용여부'].value_counts().plot.pie(labels = labels
                    ,colors= ['#98DBC6', '#FFCCBB'] #색깔 지정 (색깔 코드 인터넷 검색)
                    ,startangle = 60                #시작 각도 설정
                    ,shadow = True                  #그림자 생성
                    ,autopct = '%1.1f%%'            # 퍼센테이지 소숫점 설정
                    ,explode = (0,0.1)              # 한 파트 톡 튀어 나오게 만들기 (꼭 파트의 수와 동일해야함 안그럼 오류 EG)4개로 구분되면 (0, 0, 0, 0.1)로 기재)
                    ,figsize=(5, 5)                 # 그래프 사이즈 설정 
                    ,fontsize = 15)                 # 글씨 크기 설정
plt.title('경기도 실내 지진해일대피소 내진적용여부', fontsize=20)   #제목명, 제목 사이즈 설정
pylab.ylabel('')    # y축 설명 없애기 

![](bk3)

### 배경지식 
- 실외대피소: 지진 또는 해일이 발생할 경우 실외대피소에서 집합 후 안내 방송에 따라 다같이 이동하기 위한 용도
- 실내대피소: 집합 또는 이재민 수용을 위한 임시주거시설로 이용

![](bk3.png)

In [ ]:
labels=['미적용', '적용']
shelter_G[(shelter_G['지진해일대피소유형구분']=='학교')|(['지진해일대피소유형구분']=='체육관')]['내진적용여부'].value_counts().plot.pie(labels = labels
                    ,colors= ['#98DBC6', '#FFCCBB'] #색깔 지정 (색깔 코드 인터넷 검색)
                    ,startangle = 60                #시작 각도 설정
                    ,shadow = True                  #그림자 생성
                    ,autopct = '%1.1f%%'            # 퍼센테이지 소숫점 설정
                    ,explode = (0,0.1)              # 한 파트 톡 튀어 나오게 만들기 (꼭 파트의 수와 동일해야함 안그럼 오류 EG)4개로 구분되면 (0, 0, 0, 0.1)로 기재)
                    ,figsize=(5, 5)                 # 그래프 사이즈 설정 
                    ,fontsize = 15)                 # 글씨 크기 설정
plt.title('학교, 체육관 지진해일대피소 내진적용여부', fontsize=20)   #제목명, 제목 사이즈 설정
pylab.ylabel('')    # y축 설명 없애기 

## 2. 지진해일 대피소 구분 분석
- 지진해일대피소 vs 지진대피소

In [ ]:
len(shelter_G[shelter_G['지진해일대피소구분']=='지진대피소'])

In [ ]:
labels=['지진대피소', '지진해일대피소']
shelter_G['지진해일대피소구분'].value_counts().plot.pie(labels = labels
                    ,colors= ['#98DBC6', '#FFCCBB']
                    ,startangle = 60
                    ,shadow = True
                    ,autopct = '%1.1f%%'
                    ,explode = (0,0.1) 
                    ,figsize=(5, 5)
                    ,fontsize = 15)
plt.title('경기도 지진해일대피소구분', fontsize=20)
pylab.ylabel('')

## 4. 지진해일대피소유형구분 분석

In [ ]:
shelter_G['지진해일대피소유형구분'].unique()

In [ ]:
labels=['공원', '운동장', '학교', '기타', '체육관']
shelter_G['지진해일대피소유형구분'].value_counts().plot.pie(labels = labels
                    ,colors= ['#98DBC6','#5BC8AC','#E6D72A','#F18D9E', '#FFCCBB']
                    ,startangle = 60
                    ,shadow = True
                    ,autopct = '%1.1f%%'
                    ,explode = (0.1,0,0,0,0) 
                    ,figsize=(5, 5)
                    ,fontsize = 15)
plt.title('경기도 지진해일대피소유형구분', fontsize=20)
pylab.ylabel('')

## 5. 지진해일대피소운영상태 분석

In [ ]:
shelter_G['지진해일대피소운영상태'].unique()

In [ ]:
labels=['운영중', '비운영중']
shelter_G['지진해일대피소운영상태'].value_counts().plot.pie(labels = labels
                    ,colors= ['#98DBC6', '#FFCCBB']
                    ,startangle = 60
                    ,shadow = True
                    ,autopct = '%1.1f%%'
                    ,explode = (0.1,0) 
                    ,figsize=(5, 5)
                    ,fontsize = 15)
plt.title('경기도 지진해일대피소운영상태', fontsize=20)
pylab.ylabel('')

## 6. 지역별 대피소 수

In [ ]:
shelter_G['area'].unique()

In [ ]:
labels=['서울특별시', '경기도', '인천광역시']
shelter_G['area'].value_counts().plot.pie(labels = labels
                    ,colors= ['#98DBC6','#F18D9E', '#FFCCBB']
                    ,startangle = 60
                    ,shadow = True
                    ,autopct = '%1.1f%%' 
                    ,explode = (0.1,0, 0) 
                    ,figsize=(5, 5)
                    ,fontsize = 15)
plt.title('경기도 지진해일대피소운영상태', fontsize=20)
pylab.ylabel('')

## 7. 지역별 대피소 지도 만들기 (HELP)
- 처음에는 전국 데이터를 한번에 찍어보려 했으나 데이터가 너무 많아 지도가 뜨지 않음 
- 8도로 나누어 시도 -> 서울, 인천, 경기 3곳 모두 데이터 양이 많아 지도 안뜸 


## 서울 지도

In [ ]:
shelter_seoul = shelter_G[shelter_G['area']=='서울특별시']
# map = folium.Map(location=[35.9078, 127.7669], zoom_start = 6, tiles ='Mapbox bright')
# fg = folium.FeatureGroup(name = '서울지진해일대피소')
# for lat, lon, name in zip(shelter_seoul['위도'], shelter_seoul['경도'], shelter_seoul['지진해일대피소명']):
#     fg.add_child(folium.Marker(location=[lat, lon], popup=(folium.Popup(name)), icon=folium.Icon(icon = 'cloud', color= 'blue', icon_color = 'white')))
# map.add_child(fg)
# map.save(outfile='서울.html')

## 인천광역시 지도

In [ ]:
shelter_incheon = shelter_G[shelter_G['area']=='인천광역시']
# map = folium.Map(location=[35.9078, 127.7669], zoom_start = 6, tiles ='Mapbox bright')
# fg = folium.FeatureGroup(name = '인천지진해일대피소')
# for lat, lon, name in zip(shelter_incheon['위도'], shelter_incheon['경도'], shelter_incheon['지진해일대피소명']):
#     fg.add_child(folium.Marker(location=[lat, lon], popup=(folium.Popup(name)), icon=folium.Icon(icon = 'cloud', color= 'blue', icon_color = 'white')))
# map.add_child(fg) 
# map.save(outfile='인천.html')

## 경기도 지도

In [ ]:
shelter_ggi = shelter_G[shelter_G['area']=='경기도']
# map = folium.Map(location=[35.9078, 127.7669], zoom_start = 6, tiles ='Mapbox bright')
# fg = folium.FeatureGroup(name = '경기지진해일대피소')
# for lat, lon, name in zip(shelter_ggi['위도'], shelter_ggi['경도'], shelter_ggi['지진해일대피소명']):
#     fg.add_child(folium.Marker(location=[lat, lon], popup=(folium.Popup(name)), icon=folium.Icon(icon = 'cloud', color= 'blue', icon_color = 'white')))
# map.add_child(fg) 
# map.save(outfile='경기.html')

## 8. 지역별 인구수 대비 최대수용가능 인원수 비교

- 서울, 인천: 구 단위
- 경기도: 시 단위 
- (인구 통계가 위와 같은 단위로 분리되어있기 때문)

### 과정
1. KOSIS에서 서울, 인천, 경기도 인구 통계 불러옴 

2. 서울특별시, 인천광역시, 경기도 행 제거 (각 구 또는 시 인구 수의 합)

3. 서울 중구, 인천 중구 합쳐지는 문제 발생 
    - 인구 통계 테이블을 각 지역별로 분리하여 따로 저장 
    - 구 또는 시 이름 앞 '서울특별시', '인천광역시', '경기도' 붙여줌 
    - 다시 하나의 테이블로 합침

4. 인구 통계 테이블의 지역명과 대피소 테이블의 '제공기관명'과 분류가 동일해 '제공기관명'을 기준으로 LEFT JOIN 
    - 인구 통계 테이블에는 있지만 대피소 테이블에는 없는 지역 (=대피소 데이터가 없는 지역) 찾음 
    - 공공데이터 포털에 데이터 요청 
    - 국민재난안전포털에서 데이터 긁어옴

5. 제공기관명(지역명)으로 groupby한 후, 최대수용인원수를 sum 해서 지역별 최대수용인원수 컬럼 생성 

6. '지역 인구수 - 지역 최대수용인원수'를 'diff'컬럼에 저장 

7. diff가 마이너스면 '불충분', 플러스면 '충분'으로 지정


### 데이터 없는 지역 
- 서울특별시 용산구 
- 서울특별시 성동구
- 서울특별시 동대문구
- 서울특별시 성북구
- 서울특별시 도봉구
- 서울특별시 마포구
- 서울특별시 구로구
- 서울특별시 강동구
- 경기도 용인시
- 경기도 안양시
- 경기도 연천군

In [ ]:
ppl = pd.read_csv('population.csv', encoding = "CP949")

# http://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_1B040A3

In [ ]:
ppl.head()

In [ ]:
len(ppl)

In [ ]:
ppl[(ppl['행정구역(시군구)별']=='행정구역(시군구)별')|(ppl['행정구역(시군구)별']=='서울특별시')|(ppl['행정구역(시군구)별']=='인천광역시')|(ppl['행정구역(시군구)별']=='경기도')]

In [ ]:
ppl = ppl.drop([ppl.index[0], ppl.index[1], ppl.index[27], ppl.index[38]])

In [ ]:
ppl

In [ ]:
# 분석 쉽도로 컬럼명 변경 
ppl = ppl.rename(columns={'행정구역(시군구)별': 'sigungu', '2019. 09': 'nop'}) #nop: number of people

In [ ]:
#.iloc으로 [25번째 열까지, 모든 컬럼] 잘라내서 '서울' 테이블로 지정 
서울 = ppl.iloc[:25, :].copy()

In [ ]:
# 서울 구 이름 앞에 서울특별시 붙여넣기
서울['sigungu'] = '서울특별시' + ' ' + 서울.sigungu.str[0:] #future warning 무시 -> 해결방법을 찾아야하지만....ㅎ 

In [ ]:
서울

In [ ]:
인천 = ppl.iloc[25:35, :]

In [ ]:
인천['sigungu'] = '인천광역시' + ' ' + 인천.sigungu.str[0:]

In [ ]:
인천

In [ ]:
경기도 = ppl.iloc[35:, :]   #.drop(ppl.index[35:42]).drop(ppl.index[43:45]).drop(ppl.index[49:54]).drop(ppl.index[62:65])

In [ ]:
경기도['sigungu'] = '경기도' + ' ' + 경기도.sigungu.str[0:]

In [ ]:
경기도

In [ ]:
# 수원 = ppl.iloc[35:39, :]
# 수원['sigungu'] = '경기도' + ' ' + '수원시'+' ' + 수원.sigungu.str[0:]

In [ ]:
# 성남 = ppl.iloc[39:42, :]
# 성남['sigungu'] = '경기도' + ' ' + '성남시'+' ' + 성남.sigungu.str[0:]

In [ ]:
# 안양 = ppl.iloc[43:45, :]
# 안양['sigungu'] = '경기도' + ' ' + '안양시'+' ' + 안양.sigungu.str[0:]

In [ ]:
# 안산 = ppl.iloc[49:51, :]
# 안산['sigungu'] = '경기도' + ' ' + '안산시'+' ' + 안산.sigungu.str[0:]

In [ ]:
# 고양 = ppl.iloc[51:54, :]
# 고양['sigungu'] = '경기도' + ' ' + '고양시'+' ' + 고양.sigungu.str[0:]

In [ ]:
# 용인 = ppl.iloc[62:65, :]
# 용인['sigungu'] = '경기도' + ' ' + '용인시'+' ' + 용인.sigungu.str[0:]

In [ ]:
ppl_final = pd.concat([서울, 인천, 경기도], ignore_index=True)

In [ ]:
len(ppl_final)

In [ ]:
# 제공기관명이 시이름 + 구이름 으로 되어 있어서 이 컬럼을 기준으로 join 할 예정
shelter_G.제공기관명.unique()

In [ ]:
# 인구테이블과 join하기 위해 필요한 두개 컬럼만 shelter_ppl이라는 테이블로 저장
shelter_ppl = shelter_G[['최대수용인원수', '제공기관명']]

In [ ]:
# 제공기관명으로 groupby해서 그 지역 내 모든 대피소 최대수용인원수 더하기
shelter_capacity_temp= pd.DataFrame(shelter_ppl.groupby('제공기관명')['최대수용인원수'].sum())

In [ ]:
shelter_capacity_temp

In [ ]:
find_missing_area = ppl_final.merge(shelter_capacity_temp, how='left', left_on=['sigungu'], right_on=['제공기관명'])

In [ ]:
find_missing_area[find_missing_area['최대수용인원수'].isnull()] #뭐죠? 왜죠? 왜 없죠?

In [ ]:
shelter_G[shelter_G['제공기관명'] == '인천광역시 남동구']

In [ ]:
shelter_G[shelter_G['소재지도로명주소'].str[0:9]=='인천광역시 남동구']

In [ ]:
# join 하기
shelter_capacity = shelter_capacity_temp.merge(ppl_final, how='left', left_on=['제공기관명'], right_on=['sigungu'])

In [ ]:
shelter_capacity

In [ ]:
shelter_capacity.columns

In [ ]:
shelter_capacity.sample(10)

In [ ]:
# 분석 용이하게 하기 위해 컬럼명 변경 
shelter_capacity.rename(columns={'최대수용인원수':'max'}, inplace=True)

In [ ]:
len(shelter_capacity)

In [ ]:
# max 컬럼이 float64이기때문에 정확한 비교를 위해 타입 변경
shelter_capacity['nop'] = shelter_capacity['nop'].astype('float64')

In [ ]:
shelter_capacity[shelter_capacity['nop']>shelter_capacity['max']]

In [ ]:
# 두 컬럼의 차를 diff 컬럼에 저장
shelter_capacity['diff'] = shelter_capacity['max']-shelter_capacity['nop']

In [ ]:
shelter_capacity['diff'].max()

In [ ]:
shelter_capacity['diff'].min()

In [ ]:
shelter_capacity[shelter_capacity['diff']==shelter_capacity['diff'].max()]

In [ ]:
shelter_capacity[shelter_capacity['sigungu'] =='인천광역시 중구']

In [ ]:
# 값이 마이너스면 불충분, 플러스면 충분으로 지정
shelter_capacity['capacity'] = shelter_capacity['diff'].apply(lambda x: '불충분' if x<0 else '충분' )

In [ ]:
len(shelter_capacity[shelter_capacity['nop']>shelter_capacity['max']])

In [ ]:
shelter_capacity[shelter_capacity['nop']==shelter_capacity['nop'].max()]

In [ ]:
shelter_capacity.sort_values('nop', ascending = False).head(5)

In [ ]:
shelter_capacity.sort_values('max', ascending = False).head(5)

In [ ]:
line_chart1 = plt.plot(shelter_capacity['nop'])
line_chart2 = plt.plot(shelter_capacity['max'])
plt.title('인구수 대비 지역내 대피소 최대수용인원')
plt.xlabel('시군')
plt.ylabel('인구수')
plt.legend(['지역인구수', '최대수용인원수'], loc = 1)
plt.show()

In [ ]:
labels=['불충분', '충분']
shelter_capacity['capacity'].value_counts().plot.pie(labels = labels
                    ,colors= ['#98DBC6', '#FFCCBB']
                    ,startangle = 60
                    ,shadow = True
                    ,autopct = '%1.1f%%'
                    ,explode = (0.1,0) 
                    ,figsize=(5, 5)
                    ,fontsize = 15)
plt.title('인구수 대비 지역내 대피소 최대수용인원', fontsize=20)
pylab.ylabel('')

## 트러블 슈팅 
- 지진해일대피소 데이터가 없는 시, 구가 있다 	
    - 공공데이터 포털에 요청 
    - 국민재난안전포탈에서 끌어옴 
- 인천 중구, 서울 중구, 경기도 광주, 광주광역시와 같이 지역명 혼동 
    - 구 이름 앞에 시 이름을 붙여 구분 후 분석
- API로 위도 경도 결측치 채우기 
    - KAKAO, LacationIQ 둘다 해도 안나오는 데이터 있음 	
        - 노가다로 채움 
- 서울, 인천, 경기 각 지역 대피소 수가 많아서 지도가 안돌아감
    - 방법이 필요
- 세종시 데이터에는 중복 데이터가 있음
    - 중복 데이터 확인 후 제거 
    - 중복 이유 문의 예정 
    

### 역할분배
- 지역별로 나누어 분석중 


### 앞으로 더 해볼 것 (hopes & dreams)
- 대피소에서 더 나아가 '행정안전부_이재민임시주거시설정보'데이터를 분석하여 실내대피소와 임시주거시설 비교
- 인구수 대비 최대수용인원 부족하면 색깔이 진하게 히트맵으로 시각화하여 문제가 심각한 지역 찾기
- 지진 데이터를 보며 지진이 자주 일어나는 지역을 찾아보고 집중 분석 
- 지진대피소와 지진해일대피소의 구분 등 더 많은 배경지식 조사
- 구 또는 시 단위가 아닌 동 단위로 인구 수 대비 최대수용가능 인원수 분석 
- 개선 방안 모색 
     - 대피소 불충분 지역에 대피소를 증축한다면 위치 최적화 (입지 분석)
     - 현재 위치 기반 최단거리 대피소 추천 
     - 재난 위치에 따른 대피소 추천 
- 8월 18일 마감인 '서울특별시 빅데이터 캠퍼스 공모전' 준비 

### 동기 & 선미님께 받고 싶은 도움 
- 데이터가 많아서 지도가 안돌아가는데 어떡하나요? 
- 최적 위치 선정을 하고 싶은데 도움이 될만한 강의 또는 글이 있을까요?
- 데이터가 없는 지역 데이터 요청했는데 이유도 안말해주고 거절당했어요....안전처에서 재난소 위치는 가져올수 있지만 데이터가 많이 부족한데 이거 어떻게 하면 좋을까요?
- 피드백 주세요~~

# 수민_지도수정

### 서울

In [ ]:
shelter_seoul.shape

In [ ]:
shelter_seoul['위도'] = shelter_seoul['위도'].replace(np.nan, 0, regex = True)
shelter_seoul['경도'] = shelter_seoul['경도'].replace(np.nan, 0, regex = True)

In [ ]:
coordinates_seoul = shelter_seoul[['위도', '경도']]
coordinates_seoul.head()

In [ ]:
coordinates_seoul.isnull().sum()

## 30분 이내 도착 수용가능 범위

In [ ]:
# 지도 준비

map = Map(location=[37.55, 127], zoom_start=11)

# 지도에 위치 표시

for lat, lng in coordinates_seoul.values:
    marker = folium.Circle(
        location=[lat, lng],
        radius=2_250,
        weight=1,
        color = 'rgb(0, 255, 0)',
#         opacity=0.05,
        fill_color='rgb(0, 255, 0)',
        fill_opacity=0.05,
    )
    marker.add_to(map)
map.save(outfile='서울_30분.html')

map

## 10분 이내 도착 수용가능범위

In [ ]:
# 지도 준비

map = Map(location=[37.55, 127], zoom_start=11)

# 지도에 위치 표시

for lat, lng in coordinates_seoul.values:
    marker = folium.Circle(
        location=[lat, lng],
        radius=750,
        weight=1,
        color = 'rgb(0, 255, 0)',
#         opacity=0.2,
        fill_color='rgb(0, 255, 0)',
        fill_opacity=0.2,
    )
    marker.add_to(map)
map.save(outfile='서울_10분.html')

map

### 경기

In [ ]:
shelter_ggi.shape

In [ ]:
coordinates_ggi = shelter_ggi[['위도', '경도']]

In [ ]:
# 지도 준비

map = Map(location=[37.5, 126.9], zoom_start=9)

# 지도에 위치 표시

for lat, lng in coordinates_ggi.values:
    marker = folium.Circle(
        location=[lat, lng],
        radius=2_250,
        weight=1,
        fill_color='#F00',
        fill_opacity=0.3,
    )
    marker.add_to(map)

map

### 인천

In [ ]:
shelter_incheon.shape

In [ ]:
coordinates_incheon = shelter_incheon[['위도', '경도']]

In [ ]:
# 지도 준비

map = Map(location=[37.4, 126.4], zoom_start=10)

# 지도에 위치 표시

for lat, lng in coordinates_incheon.values:
    marker = folium.Circle(
        location=[lat, lng],
        radius=1_000,
        weight=1,
        fill_color='#F00',
        fill_opacity=0.3,
    )
    marker.add_to(map)

map